In [ ]:
# modules for research report
from datascience import *
import numpy as np
import random
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# module for YouTube video
from IPython.display import YouTubeVideo

# 2016 Global Ecological Footprint – Prediction Examples

This dataset was originally published on [Kaggle.com](https://www.kaggle.com/footprintnetwork/ecological-footprint). It has been
cleaned for your convenience: all missing values have been removed, and low-quality observations and variables have been filtered
out. A brief descriptive summary of the dataset, originally posted on the dataset's Kaggle landing page, is provided below.

### Summary

> The ecological footprint measures the ecological assets that a given population requires to produce the natural resources it
 consumes (including plant-based food and fiber products, livestock and fish products, timber and other forest products, space for
 urban infrastructure) and to absorb its waste, especially carbon emissions. The footprint tracks the use of six categories of
 productive surface areas: cropland, grazing land, fishing grounds, built-up (or urban) land, forest area, and carbon demand on land.

> A nation’s biocapacity represents the productivity of its ecological assets, including cropland, grazing land, forest land, fishing
 grounds, and built-up land. These areas, especially if left unharvested, can also absorb much of the waste we generate, especially
 our carbon emissions.
 
> Both the ecological footprint and biocapacity are expressed in global hectares — globally comparable, standardized hectares with
 world average productivity.

> If a population’s ecological footprint exceeds the region’s biocapacity, that region runs an ecological deficit. Its demand for the
 goods and services that its land and seas can provide — fruits and vegetables, meat, fish, wood, cotton for clothing, and carbon dioxide
 absorption — exceeds what the region’s ecosystems can renew. A region in ecological deficit meets demand by importing, liquidating its
 own ecological assets (such as overfishing), and/or emitting carbon dioxide into the atmosphere. If a region’s biocapacity exceeds its
 ecological footprint, it has an ecological reserve.
 
> The ecological footprint measure was conceived by Mathis Wackernagel and William Rees at the University of British Columbia. Ecological
 footprint data was provided by the Global Footprint Network.
 
### Data Description

This dataset consists of two tables stored in the `data` folder:
1. `ecological-info` provides information on a countries' ecological footprint.
2. `economic-info` contains a limited amount of economic and geographical data for all countries listed in `ecological-info`.

A description of each table's variables is provided below:

`ecological-info`:
* `Country`: country name
* `Grazing Footprint`: grazing footprint in standardized hectares
* `Cropland Footprint`: cropland footprint in standardized hectares
* `Forest Footprint`: forest footprint in standardized hectares
* `Fish Footprint`: fishing footprint in standardized hectares
* `Carbon Footprint`: carbon footprint in standardized hectares
* `Total Ecological Footprint`: total ecological footprint, standardized
* `Grazing Land`: grazing land in standardized hectares
* `Cropland Land`: cropland land in standardized hectares
* `Forest Land`: forest land standardized hectares
* `Fishing Water`: fishing water in standardized hectares
* `Urban Land`: urban land in standardized hectares
* `Total Biocapacity`: total biocapacity in standardized hectares
* `Biocapacity Deficit or Reserve`: difference between total biocapacity and total ecological footprint
* `Earths Required`: The number of planet Earths required if the average human consumed as much the average citizen of this country

`econ-info`:
* `Country`: country name
* `Region`: country region
* `Continent`: continent where country is located
* `Log Population (millions)`: log transformed population in millions (2016)
* `HDI`: Human development index (HDI)
* `HDI Rating`: Binarized HDI ("High or Very High": HDI > 0.7; "Medium or Low": HDI < 0.7)
* `Log GDP per Capita`: log transformed gross domestic product per capita, in USD


The tables are loaded in the code cells below. Take some time to explore them!

In [ ]:
# load the economic data
economic_data = Table.read_table('economic-info')
economic_data

In [ ]:
# load the ecological data
ecological_data  = Table.read_table('ecological-info')
ecological_data

## Prediction #1

We'll fit a linear regression to our ecological footprint data to determine if countries' 'Biocapacity Deficit or
Reserve' can be accurately predicted based on their 'Log GDP per Capita'. We fit the linear regression model
using the methods taught in class, and then we evaluate whether our model accurately reflects reality by studying our
prediction's residuals.

In [ ]:
# define the functions needed to fit the linear regression
def standard_units(array_of_num):
    """Converts a numeric array to standard units."""
    standardized_array = (array_of_num - np.mean(array_of_num)) / np.std(array_of_num)
    return standardized_array

def correlation(tbl, var1, var2):
    """Computes the correlation coefficient of two variables"""
    r = np.mean(standard_units(tbl.column(var1)) * standard_units(tbl.column(var2)))
    return r

def slope(tbl, x, y):
    """Compute the slope of the regression line. x is the independent variable, y the depenent variable."""
    r = correlation(tbl, x, y)
    m = r * np.std(tbl.column(y)) / np.std(tbl.column(x))
    return m

def intercept(tbl, x, y):
    """Compute the intercept of the regression line. x is the independent variable, y the depenent variable."""
    b = np.mean(tbl.column(y)) - slope(tbl, x, y) * np.mean(tbl.column(x))
    return b

def linear_fit(tbl, x, y):
    m = slope(tbl, x, y)
    b = intercept(tbl, x, y)
    return m * tbl.column(x) + b

In [ ]:
# join the tables
full_data = economic_data.join('Country', ecological_data)

In [ ]:
# compute the correlation between the two variables - this would go in the Exploration section as a quantitative plot
correlation(full_data, 'Log GDP per Capita', 'Biocapacity Deficit or Reserve')

In [ ]:
# scatter plot of the variables - this would go in the Exploration section as a quantitative plot
full_data.scatter('Log GDP per Capita', 'Biocapacity Deficit or Reserve')

In [ ]:
# fit the linear regression, store results in table
linear_regression_results = Table().with_columns(
    'Log GDP per Capita', full_data.column('Log GDP per Capita'),
    'Biocapacity Deficit or Reserve', full_data.column('Biocapacity Deficit or Reserve'),
    'Fitted Values', linear_fit(full_data, 'Log GDP per Capita', 'Biocapacity Deficit or Reserve')
)

# plot the fitted results agains the true Biocapacity Deficit and Rerserves
linear_regression_results.scatter('Log GDP per Capita')

In [ ]:
# compute the residuals of our linear regression model
linear_regression_results = linear_regression_results.with_column(
    'Residuals', linear_regression_results.column(1) - linear_regression_results.column(2)
)
linear_regression_results.scatter('Log GDP per Capita', 'Residuals')

These variables are very weakly, negatively correlated (r = -0.223). Further, the
scatter plots of the fitted values and of the residuals suggest a non-linear relationship between these two variables.
As the log GDP per capita of the countries in our sample increase, the values of Biocapacity Deficit or Reserve begin to fluctuate
erratically about zero. A more complex model is needed to accurately explain the variation in a countries Biocapacity Reserve
or Deficit.

## Prediction #2

Now that we've covered how to report a "failed" application of linear regression – that is, when you find that the variables that you're
interested in aren't linearly related - we can look at what a successful application of this method might look like.

Let's turn our attention to the relationship between the Log GDP per capita of a country and its
[Human Development Index](http://hdr.undp.org/en/content/human-development-index-hdi) (HDI). In
particular, we're interested in whether we can predict the HDI of a country not contained in our sample based on it's log GDP per
capita. These two variables are numerical, so we will apply a linear regression in an attempt to solve this problem.

In [ ]:
# first, let's explore the relationship between these two variables - this would go in the Exploration section as a quantitative plot
full_data.scatter('Log GDP per Capita', 'HDI')

In [ ]:
# what is the correlation between these two variables - this would go in the Exploration section as a quantitative plot
correlation(full_data, 'Log GDP per Capita', 'HDI')

In [ ]:
# fit the linear regression, store results in table
linear_regression_results = Table().with_columns(
    'Log GDP per Capita', full_data.column('Log GDP per Capita'),
    'HDI', full_data.column('HDI'),
    'Fitted Values', linear_fit(full_data, 'Log GDP per Capita', 'HDI')
)

# plot the fitted results agains the true Biocapacity Deficit and Rerserves
linear_regression_results.scatter('Log GDP per Capita')

In [ ]:
# compute the residuals of our linear regression model
linear_regression_results = linear_regression_results.with_column(
    'Residuals', linear_regression_results.column(1) - linear_regression_results.column(2)
)
linear_regression_results.scatter('Log GDP per Capita', 'Residuals')
linear_regression_results.scatter('HDI', 'Residuals')

*(These diagnostics suggest that the relationship is linear. We can go ahead and interpret these results.)*

In [ ]:
# what is the slope of this linear regression?
slope(full_data, 'Log GDP per Capita', 'HDI')

In [ ]:
# prediction function
def predict_hdi(log_gdp):
    m = slope(full_data, 'Log GDP per Capita', 'HDI')
    b = intercept(full_data, 'Log GDP per Capita', 'HDI')
    return m * log_gdp + b

In [ ]:
# want to predict the HDI of the Republic of Ireland
log_gdp_ireland = np.log(71921.72) # taken from World Bank
ireland_prediction =  predict_hdi(log_gdp_ireland)
ireland_prediction

In [ ]:
# measure error
actual_HDI_2015 = 0.923 # taken from UN report in 2016
error = actual_HDI_2015 - ireland_prediction
error

Based on the scatter plots of the "Log GDP per Capita" versus "HDI" and "Log GDP per Capita" vs "Residuals", there is evidence to
suggest that the relationship between the variables of interest is linear. Our linear regression suggests that for every unit
increase in log GDP per capita, the average country's HDI increases by 0.097. One caveat of this predictive model is that the HDI
has a range of 0 to 1. Some countries with extremely small or large log GDP per capitas might produce estimated HDIs not contained
in this range.

Given our positive results, we used this method to predict the HDI of a country not contained in a dataset: The Republic of Ireland.
The GDP per capita of this country was reported to be \$71,921.72 by the
[World Bank](https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD?locations=IE-GB). We computed the log of this value, and used it
to estimate its HDI. Our linear regression procedure estimated an HDI of 0.935, which is very close to its actual value of 0.923 in 2016.
The true HDI was reported by the
[United Nation's Human Development Report of 2016](http://hdr.undp.org/sites/default/files/2016_human_development_report.pdf). This
provides further evidence that our linear regression provides a reasonable approximation of the true relationship between a country's
log GDP per capita and its HDI.